# Sentiment Analysis with Hugging Face

Hugging Face is an open-source and platform provider of machine learning technologies. You can use install their package to access some interesting pre-built models to use them directly or to fine-tune (retrain it on your dataset leveraging the prior knowledge coming with the first training), then host your trained models on the platform, so that you may use them later on other devices and apps.

Please, [go to the website and sign-in](https://huggingface.co/) to access all the features of the platform.

[Read more about Text classification with Hugging Face](https://huggingface.co/tasks/text-classification)

The Hugging face models are Deep Learning based, so will need a lot of computational GPU power to train them. Please use [Colab](https://colab.research.google.com/) to do it, or your other GPU cloud provider, or a local machine having NVIDIA GPU.

In [1]:
# Install required libraries
%pip install torch
%pip install transformers
%pip install datasets
%pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 61.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 102.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 857.6 kB/s eta 0:00:00


## Application of Hugging Face Text classification model Fune-tuning

Find below a simple example, with just `3 epochs of fine-tuning`.

Read more about the fine-tuning concept : [here](https://deeplizard.com/learn/video/5T-iXNNiwIs#:~:text=Fine%2Dtuning%20is%20a%20way,perform%20a%20second%20similar%20task.)

In [ ]:
import os
import numpy as np
import pandas as pd
from datasets import load_dataset
from datasets import load_metric
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from collections import Counter
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
from transformers import TrainingArguments
from transformers import AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding
from transformers import Trainer
from huggingface_hub import notebook_login
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Disabe W&B
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
# Load the train dataset
train_df = pd.read_csv('/content/drive/MyDrive/NLP-Vaccination-Sentiment-Analysis LP5/data/Train.csv')
train_df.head()

In [ ]:
# Load the test dataset
test_df = pd.read_csv('/content/drive/MyDrive/NLP-Vaccination-Sentiment-Analysis LP5/data/Test.csv')
test_df.head()

# Exploratory Data Analysis: EDA

### Column Information of The Datasets

In [ ]:
train_df.info()

- The train DataFrame contains a total of 10,001 entries (rows), and four columns: 'tweet_id', 'safe_text', 'label', and 'agreement'.
- The 'tweet_id' and 'safe_text' columns are of type 'object', typically representing strings. The 'label' and 'agreement' columns are of type 'float64', indicating floating-point numbers.
- The 'label' and 'agreement' columns contain missing values.

In [ ]:
test_df.info()

- The test DataFrame consists of 5,177 entries (rows), and two columns: 'tweet_id' and 'safe_text'.
- Both columns, 'tweet_id' and 'safe_text', are of type 'object', representing strings.
- The 'safe_text' column has a missing value.

In [ ]:
# Checking the shape of the train dataset
train_df.shape

The train dataset as earlier mentioned has 1001 rows and 4 columns.

In [ ]:
# Checking the shape of the train dataset
test_df.shape

The test dataset has 5177 rows and 2 columns. The dataset lacks the 'label' and the 'safe_test' columns.

### Checking for and Handling Missing Values

#### Train Data

In [ ]:
# Checking for missing values in the train dataset
train_df.isnull().sum()

In [ ]:
# Drop The Rows With Missing Data
train_df = train_df.dropna()

# Confirm if the missing values have been handled
train_df.isnull().sum()

#### Test Data

In [ ]:
# Checking for missing values in the test dataset
test_df.isnull().sum()

In [ ]:
# Drop The Rows With Missing Data
test_df = test_df.dropna()

# Confirm if missing values have been handled
test_df.isnull().sum()

All missing Values in the Train and Test Datasets have been handled.

### Visualizing The Distribution of Data In The Different Datasets

#### i. Distribution of The 'label' column

In [ ]:
# Replace numeric sentiment labels with descriptive words
label_mapping = {-1: 'Negative', 0: 'Neutral', 1: 'Positive'}
train_df['label'] = train_df['label'].map(label_mapping)

# Class Distribution in 'label' column with 'viridis' color palette
plt.figure(figsize=(8, 6))
sns.countplot(data=train_df, x='label', palette='viridis')
plt.title('Lalel Distribution in Train Dataset')
plt.xlabel('Sentiment Labels')
plt.ylabel('Count')
plt.show()

The plot visualizes the distribution of sentiment labels ('Negative,' 'Neutral,' 'Positive') in the training dataset.
From the plot, we can observe the following insights:
- The 'Neutral' sentiment appears to be the most frequent category, followed by 'Positive,' and 'Negative' sentiments.
- The sentiment classes seem to be relatively balanced, with 'Neutral' having the highest count and 'Negative' having the lowest count.
- This balance suggests that the dataset does not suffer from severe class imbalance issues, which is beneficial for model training.

### ii. Exploration of Text Data (Tweet_Length)

In [ ]:
# Calculate and visualize the length of tweets
train_df['tweet_length'] = train_df['safe_text'].apply(len)

plt.figure(figsize=(8, 6))
sns.histplot(train_df['tweet_length'], bins=50, palette='viridis')
plt.title("Distribution of Tweet Length (Character Count)")
plt.xlabel("Tweet Length")
plt.ylabel("Count")
plt.show()

The plot explores the distribution of tweet lengths in terms of character count. Insights from this plot include:
- The distribution is left-skewed, with most tweets having relatively longer character counts.
- There is a peak in the distribution around the 100-150 character mark.
- This suggests that a substantial portion of tweets related to vaccination topics is relatively concise, but there is also a presence of shorter tweets.
- These insights can be valuable for further data preprocessing and model development. Understanding the distribution of tweet lengths can help in deciding the appropriate text preprocessing techniques and model architectures for handling different text lengths effectively.For instance, models should be capable of handling both longer and shorter texts effectively, considering the variability in tweet lengths within the dataset.

### iii. Exploration of Text Data

In [ ]:
# Plot the distribution of agreement
plt.figure(figsize=(8, 6))
sns.histplot(train_df['agreement'], bins=30)
plt.title("Distribution of Agreement")
plt.xlabel("Agreement Percentage")
plt.ylabel("Count")
plt.show()

The plot visualizes the distribution of the agreement percentages in the training dataset. The agreement percentage indicates the percentage of reviewers who agreed on the sentiment label for a given tweet.
Key insights from this plot:
- Many tweets in the dataset have a consensus among reviewers regarding their sentiment labels.
- However, there is also a presence of tweets with lower agreement percentages, indicating potential disagreement among reviewers on sentiment labeling.

### Visualize Word Cloud From  The Text Contained in The Tweets

In [ ]:
# Generate a word cloud from the tweet text
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(' '.join(train_df['safe_text']))

plt.figure(figsize=(11, 10))
plt.imshow(wordcloud, interpolation='bilinear')
plt.title("Word Cloud of Tweet Text")
plt.axis('off')
plt.show()

The plot presents a word cloud generated from the tweet text in the training dataset.
Insights from this word cloud:
- The most frequent words in the tweet text are displayed in larger fonts, making them easily identifiable.
- Common words related to vaccination topics, and public health such as vaccine, vaccinate, vaccination are more prominent in the word cloud.
- This visualization provides a qualitative view of the textual content in the dataset, highlighting the importance of certain words or phrases.

In [ ]:
# Extract hashtags from the 'safe_text' column
hashtags = train_df['safe_text'].str.findall(r'#\w+')

# Flatten the list of lists into a single list of hashtags
hashtags = [item for sublist in hashtags for item in sublist]

# Count the frequency of each hashtag
hashtag_freq = Counter(hashtags)

# Get the top 10 hashtags
top_10_hashtags = dict(hashtag_freq.most_common(10))

# Create a bar plot to visualize the top 10 hashtags with the 'viridis' color palette
plt.figure(figsize=(10, 6))
sns.barplot(x=list(top_10_hashtags.keys()), y=list(top_10_hashtags.values()), palette='viridis')
plt.xlabel('Hashtags')
plt.ylabel('Frequency')
plt.title('Top 10 Hashtags')
plt.xticks(rotation=45)
plt.show()

The bar plot reveals the top 10 hashtags that are most frequently used in the Twitter posts within the dataset
Each hashtag represents a topic or theme discussed in the Twitter posts. By examining the top hashtags, we identify the most prevalent topics of discussion among Twitter users.

# Splitting the Train Dataset Into Train and Evaluation Subsets

I manually split the training set to have a training subset ( a dataset the model will learn on), and an evaluation subset ( a dataset the model with use to compute metric scores to help use to avoid some training problems like [the overfitting](https://www.ibm.com/cloud/learn/overfitting) one ).

There are multiple ways to do split the dataset. You'll see two commented line showing you another one.

In [ ]:
# Split the train data => {train, eval}
train, eval = train_test_split(train_df, test_size=0.2, random_state=42, stratify=train_df['label'])

In [ ]:
train.head()

In [ ]:
eval.head()

In [ ]:
print(f"The new dataframe shapes: train is {train.shape}, eval is {eval.shape}")

In [ ]:
# Save splitted subsets
train.to_csv("/content/drive/MyDrive/NLP-Vaccination-Sentiment-Analysis LP5/data/train_subset.csv", index=False)
eval.to_csv("/content/drive/MyDrive/NLP-Vaccination-Sentiment-Analysis LP5/data/eval_subset.csv", index=False)

In [ ]:
# Load the dataset
dataset = load_dataset('csv',
                        data_files={'train': '/content/drive/MyDrive/NLP-Vaccination-Sentiment-Analysis LP5/data/train_subset.csv',
                        'eval': '/content/drive/MyDrive/NLP-Vaccination-Sentiment-Analysis LP5/data/eval_subset.csv'}, encoding = "ISO-8859-1")


In [ ]:
# Import the Tokenizer for RoBERTa
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment", num_labels=3)

Tokenization is the process of converting a sequence of text, such as a sentence or a document, into individual numerical units or tokens. In the context of natural language processing (NLP), these tokens are typically words or subwords. Tokenization is a crucial step in preparing text data for analysis or machine learning tasks.



In [ ]:
# Function for transforming the labels
def transform_labels(label):
    label = label['label']
    num = 0
    if label == 'Negative': # -1
        num = 0
    elif label == 'Neutral': # 0
        num = 1
    elif label == 'Positive': # 1
        num = 2

    return {'labels': num}

# Function to tokenize the text data
def tokenize_data(example):
    return tokenizer(
        example['safe_text'],
        padding='max_length',
        truncation=True,
        max_length=256
    )

# Apply tokenization to the dataset, adding 'input_ids' and 'attention_mask' columns
dataset = dataset.map(tokenize_data, batched=True)

# Transform labels to numerical values and remove unnecessary columns
remove_columns = ['tweet_id', 'label', 'safe_text', 'agreement']
dataset = dataset.map(transform_labels, remove_columns=remove_columns)

In [ ]:
# View the contents and structure of the processed train and test datasets
dataset

In [ ]:
# Access labels for the train dataset
train_labels = dataset['train']['labels']

# Access labels for the eval dataset
eval_labels = dataset['eval']['labels']

# Print the first few labels for reference
print("Train Labels:", train_labels[:20])
print("Eval Labels:", eval_labels[:20])

The labels in both datasets have been properly tokenized.

In [ ]:
# Configure the training parameters using TrainingArguments
training_args = TrainingArguments(

    # Directory where model checkpoints and logs will be saved
    "covid-19_tweets_sentiment_analysis_RoBERTa_model",

    # Number of training epochs (iterations over the dataset)
    num_train_epochs=5,

    # Load the best model at the end of training
    load_best_model_at_end=True,

    # Define evaluation strategy (evaluate at specified epochs)
    evaluation_strategy="epoch",

    # Define how often to save model checkpoints
    save_strategy="epoch",

    # Directory for logs
    logging_dir="./logs",

    # Log every N steps
    logging_steps=500

)

In [ ]:
# Loading the pretrained RoBERTa model while specifying the number of labels in our dataset for fine-tuning
model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment", num_labels=3)

In [ ]:
# Define evaluation metrics
metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
# Instantiate the training and evaluation sets
train_dataset = dataset['train'].shuffle(seed=100)
eval_dataset = dataset['eval'].shuffle(seed=100)

In this code, we are creating training and evaluation datasets from our original dataset. We shuffle the data using a random seed of 100. Shuffling is a common practice in training machine learning models to ensure that the data samples are presented to the model in a random order during each epoch. This helps prevent any potential bias in the training process.

In [ ]:
# Converting training data to PyTorch tensors and adding padding for improved training efficiency:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

This code prepares a data collator that will convert the training data into PyTorch tensors and add padding to the sequences. This is done to optimize the training process, making it more efficient and faster. Padding ensures that all sequences have the same length, which is important for efficient batch processing during training.

In [ ]:
# Instantiate the trainer
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
    tokenizer = tokenizer,
    data_collator = data_collator,
    compute_metrics = compute_metrics
)

In [ ]:
# Launch the learning process: training
trainer.train()

In [ ]:
# Reinstantiate the trainer for evaluation
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
# Launch the final evaluation
trainer.evaluate()

In [ ]:
# Login to HuggingFace Hub
notebook_login()

In [ ]:
# Push the fine-tuned model and tokenizer to HuggingFace Hub
model.push_to_hub("rasmodev/Covid-19_Sentiment_Analysis_RoBERTa_Model", create_pr=1)
tokenizer.push_to_hub("rasmodev/Covid-19_Sentiment_Analysis_RoBERTa_Model", create_pr=1)